
# 🧠 From Predictive to Prescriptive Churn Reduction with Gen AI

---

Retailers face increasing customer churn, rising acquisition costs, and fierce competition, making customer retention more critical than ever.
Despite investments in loyalty programs and CRM systems, many struggle with fragmented customer data, ineffective engagement strategies, and the inability to act in real-time—resulting in missed opportunities and declining customer lifetime value.

<div style="background: #f7f7f7; border-left: 5px solid #ff5f46; padding: 20px; margin: 20px 0; font-size: 18px;">
"A recent review of the State of Customer Churn revealed that the <b>average churn rate</b> for the entire retail sector is <b>6.58%</b>, far higher than the <b>ideal rate of 3-5%</b>."[^3]
</div>

<div style="background: #f7f7f7; border-left: 5px solid green; padding: 20px; margin: 20px 0; font-size: 18px;">
"With <b>prescriptive analytics</b> in place, decision makers can quickly extract <b>actionable knowledge</b> from the available data and define <b>effective actions</b> for the future."[^2]
</div>

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" style="width: 100px; vertical-align: middle; margin-right: 10px; float: left;" />
<br>
Liza, a Generative AI Engineer, is going to utilize Databricks AI in order to create a prescriptive system for her company to reduce customer churn. She will use the full power of her company's data platform to create tailored advertising to her at-risk customers to encourage new purchases.

[^1]: https://www.qualtrics.com/experience-management/customer/customer-churn/

[^2]: https://voziq.ai/featured/prescriptive-analytics-for-churn-reduction/

[^3]: https://www.custify.com/blog/customer-churn-guide/

[^4]: https://www.infobip.com/blog/how-generative-ai-can-help-reduce-churn

[^5]: https://www.salesforce.com/sales/analytics/customer-churn/

[^6]: https://graphite-note.com/customer-churn-prevention-predictive-analytics/

[^7]: https://churnzero.com/blog/customer-success-statistics/

[^8]: https://www.b2brocket.ai/blog-posts/ai-powered-churn-prediction



## The Demo: Generative AI for Smarter Churn Reduction  
<!-- 
<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/retail/lakehouse-churn/lakehouse-retail-c360-agent-functions.png" 
     style="float: right; margin-top: 10px; width: 700px; display: block;" /> -->

Databricks Generative AI is redefining churn management by allowing organizations to shift from reactive retention to proactive customer engagement. Unlike traditional predictive models that simply flag at-risk customers, Databricks AI Agent Systems allow the use of those predictive models in a high-value business context.  

In this demo, we show a simple use-case for Databricks to help reduce customer churn. We will create an AI agent using the following functions, defined in the subsequent cells:


1. **Customer Churn Prediction**: We will create a function that will allow our GenAI agent to determine if a customer is at risk of churn, based on the AutoML model we defined in **section 4** of this demo.
2. **Customer Order Lookup**: Then, our agent needs to be able to look up that at-risk-of-churn customer's most recent orders and also customer's demographic information in order to create personalized outreach.
3. **Personalized Customer Outreach**: Finally, our agent will use this outreach function to piece together a personalized ad outreach based on the customer's demographics and purchase history to keep them engaged and a buying customer!

By leveraging Generative AI, retailers can move beyond predicting churn to actively **preventing it—turning potential losses into fruitful customer relationships**.   



<!-- Architecture doc: https://docs.google.com/presentation/d/1YzY7jj4GVXaEUjVPFPI74qgy00mVO6rWXHwv6iz0-lM/edit#slide=id.gf7b9ec9ce5_0_1087 -->


In [0]:
# CONFIGURATION CELL, installing necessary packages for this demo.
%pip install mlflow==2.20.1 databricks-vectorsearch==0.40 databricks-feature-engineering==0.8.0 databricks-sdk==0.40.0
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup $reset_all_data=false

## Function 1: Customer Churn Prediction



In [0]:
spark.sql("DROP FUNCTION IF EXISTS churn_predictor")

spark.sql(f"""
          CREATE OR REPLACE FUNCTION churn_predictor(id STRING)
RETURNS STRING
LANGUAGE SQL
COMMENT 'This function determines whether a customer is at risk of churning. The possible value are AT RISK or NOT AT RISK of churning. Use this function when a User ID is given.'
RETURN
(
    SELECT CASE 
             WHEN ai_query(
                    'dbdemos_customer_churn_endpoint', 
                    named_struct(
                        'user_id', user_id,
                        'canal', MAX(canal),
                        'country', MAX(country),
                        'gender', MAX(gender),
                        'age_group', MAX(age_group),
                        'order_count', MAX(order_count),
                        'total_amount', MAX(total_amount),
                        'total_item', MAX(total_item),
                        'last_transaction', MAX(last_transaction),
                        'platform', MAX(platform),
                        'event_count', MAX(event_count),
                        'session_count', MAX(session_count),
                        'days_since_creation', MAX(days_since_creation),
                        'days_since_last_activity', MAX(days_since_last_activity),
                        'days_last_event', MAX(days_last_event)
                    ),
                    'STRING'
                 ) = '0' THEN 'NOT AT RISK'
             ELSE 'AT RISK'
           END
    FROM {catalog}.{db}.churn_user_features
    WHERE user_id = id GROUP BY user_id
)""")

### Example:

In [0]:
%sql
SELECT churn_predictor(
    '2d17d7cd-38ae-440d-8485-34ce4f8f3b46'
) AS prediction


##Function 2: Customer Order & Demographic Lookup

In [0]:
spark.sql(f"""
  CREATE OR REPLACE FUNCTION customer_order_lookup (
    input_user_id STRING
    COMMENT 'User ID of the customer to be searched'
  ) 
  RETURNS TABLE(
    firstname STRING,
    channel STRING,
    country STRING,
    gender INT,
    order_amount INT,
    order_item_count INT,
    last_order_date STRING,
    current_date STRING,
    days_elapsed_since_last_order INT
  )
  COMMENT "This function returns the customer details for a given customer User ID, along with their last order details. The return fields include First Name, Channel (e.g. Mobile App, Phone, or Web App/Browser ), Country of Residence, Gender, Order Amount, Item Count, and the Last Order Date (in format of yyyy-MM-dd). Use this function when a User ID is given." 
  RETURN (
    SELECT
      firstname,
      canal as channel,
      country,
      gender,
      o.order_amount,
      o.order_item_count,
      o.last_order_date,
      TO_CHAR(CURRENT_DATE(), 'yyyy-MM-dd') AS current_date,
      DATEDIFF(CURRENT_DATE(), o.last_order_date) AS days_elapsed_since_last_order
    FROM 
      {catalog}.{db}.churn_users u
    LEFT JOIN (
      SELECT
        user_id,
        amount AS order_amount,
        item_count AS order_item_count,
        TO_CHAR(creation_date, 'yyyy-MM-dd') AS last_order_date
      FROM {catalog}.{db}.churn_orders
      WHERE user_id = input_user_id
      ORDER BY creation_date DESC
      LIMIT 1
    ) o ON u.user_id = o.user_id
    WHERE
      u.user_id = input_user_id
  )
""")

### Example:

In [0]:
%sql
SELECT * FROM customer_order_lookup('2d17d7cd-38ae-440d-8485-34ce4f8f3b46')

##Function 3: Personalized Customer Outreach with Automated Marketing Copy Generation

### Define the model prompt

In [0]:
prompt = """
    Your only role is to write copy for a contemporary apparel retailer. All your responses are a piece of copy, nothing further.\n
    You have access to the following information from the customer and their last order:\n
    - firstname: the first name of the customer
    - channel: the channel of which the customer is using. This could be WEBAPP for Web app or MOBILE for Mobile app
    - country: the customer's country of residence. This field uses string country code, for example FR for France
    - gender: the customer's gender. This field uses 0 for Male and 1 for Female
    - order_amount: the total amount in US dollars of the last order
    - order_item_count: the total item of the last order
    - last_order_date: the date of the last order in format of yyyy-MM-dd
    - current_date: the current date in format of yyyy-MM-dd
    - days_elapsed_since_last_order: the number of days elapsed since the last order
    - churn: either the customer is AT RISK or NOT AT RISK at churning
    These are the rules you must respect when writing copy:
    - Start every message by greeting the customer using their first_name. Do not include a final greeting.\n
    - When the customer's country of residence is not an English-spoken country, include a second message in their native language based on their country of residence. For example, if the country is FR, write two messages: one in English, and another in French. Never explicitly mention their country of residence.\n
    - Write short copy (SMS or push notification style) when channel is PHONE and MOBILE, and long copy (email style) for canal WEBAPP. Never explicitly mention the channel.
    - Never mention a customers gender.\n
    - Consider the total amount of their last order when pushing for discount. For example, if the total amount is less than $100, push for discount. If the total amount is greater than $100, push for a free shipping offer.\n
    - Consider the total number of items in their last order when pushing for package deal. For example, if the total number of items is less than 5, push for package deal. If the total number of items is greater than 5, push for a free shipping offer.\n
    - Consider the days elapsed since the last purchased date when writing copy. For example, if the time elapsed is less than 30 days, push for discount. If the time elapsed is greater than 30 days, push for a free shipping offer.\n
    - Consider the current date when writing copy to take into account the seasonality of the country. For example, if the current season is Summer in the customer's country, promote for some products in summer and vice versa.\n
    - If the customer is at risk at churning, add urgency to the copy. Never explicitly mention the customer's churn status.\n
    - Make sure the input parameters match the order of the function.
    """

### Function definition

In [0]:
spark.sql(f"""
  CREATE OR REPLACE FUNCTION generate_marketing_copy(input_user_id STRING, firstname STRING, channel STRING, country STRING, gender INT, order_amount INT, order_item_count INT, last_order_date STRING, current_date STRING, days_elapsed_since_last_order INT, churn_status STRING)
  RETURNS STRING
  LANGUAGE SQL
  COMMENT "This function generates marketing copy for a given customer User ID. This function expects the user order information which can be found with _____, as well as their churn status which is found with ______. This function returns the marketing copy as a string. "
  RETURN (
    SELECT ai_query(
      'databricks-meta-llama-3-1-405b-instruct', 
      CONCAT(
        "{prompt}" 
        'Customer details: ', 
        TO_JSON(NAMED_STRUCT('firstname', firstname, 'channel', channel, 'country', country, 'gender', gender, 'order_amount', order_amount, 'order_item_count', order_item_count, 'last_order_date', last_order_date, 'current_date', current_date, 'days_elapsed_since_last_order', days_elapsed_since_last_order, 'churn_status', churn_status))
      )
    )
  )
  """)

### Example:

In [0]:
%sql
SELECT generate_marketing_copy(
  '2d17d7cd-38ae-440d-8485-34ce4f8f3b46',
  'Christopher',
  'WEBAPP',
  'USA',
  0,
  105,
  3,
  '2023-06-07',
  '2025-03-05',
  637,
  churn_predictor('2d17d7cd-38ae-440d-8485-34ce4f8f3b46')
)

## Next Steps

Proceed to notebook **05.2-Agent-Creation-Guide** in order to package the above functions into an implementable AI Agent with Databricks!